In [11]:
import pandas as pd
import dtale
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import plotly.graph_objects as go # for visualization
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import os
import statsmodels.api as sm

%load_ext nb_black
%load_ext blackcellmagic

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


<IPython.core.display.Javascript object>

In [12]:
df = pd.read_excel("../../../../site_info_ver_3.2.xlsx")


<IPython.core.display.Javascript object>

Check Null

In [13]:
# df.isna().sum()
df=df.dropna()
df.reset_index(inplace=True,drop=True)
len(df[df.isnull().any(axis=1)])/len(df)

0.0

<IPython.core.display.Javascript object>

Defiend X and y

In [14]:
X = df.drop(['Israelis_Count','Tourists_Count','Total',"Date"],axis=1)
targets = df[['Israelis_Count','Tourists_Count','Total']]

X_dummies = pd.get_dummies(X,'Site_Name',drop_first=True)
targets

,Israelis_Count,Tourists_Count,Total
0,33,0,33
1,23,0,23
2,12,0,12
3,5,0,5
4,14,0,14
...,...,...,...
46482,6,0,6
46483,5,0,5
46484,2,0,2
46485,5,0,5


<IPython.core.display.Javascript object>

Scaleing

In [15]:

#scale X
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_X = pd.DataFrame(scaler.fit_transform(X_dummies), columns=X_dummies.columns)
scaled_X.head()
dtale.show(scaled_X)

#scale y
scaled_targets = np.log(targets+0.01)
dtale.show(scaled_targets)

<IPython.core.display.Javascript object>

Outliers

In [16]:
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(scaled_targets.columns))]
fig = go.Figure()
fig.add_traces(data=[go.Box( 
    y=scaled_targets.iloc[:, i], 
    marker_color=c[i],
    name=scaled_targets.columns[i])
    for i in range(len(scaled_targets.columns))
    ])





<IPython.core.display.Javascript object>

In [17]:
outliers_I = scaled_targets[(scaled_targets.Israelis_Count>9.59) | (scaled_targets.Israelis_Count<0.009)]
outliers_To = scaled_targets[(scaled_targets.Total>9.6) | (scaled_targets.Total<0.69)]
print('Isrealis_count',len(outliers_I),' rows| ',len(outliers_I)/len(scaled_targets),"%")
print('Total',len(outliers_To),' rows|',len(outliers_To)/len(scaled_targets),"%")

Isrealis_count 278  rows|  0.005980166498160776 %
Total 222  rows| 0.00477552864241616 %


<IPython.core.display.Javascript object>

In [18]:
# targets_list_name = ['Israelis_Count','Total','Tourists_Count']
# idx=0
# y = scaled_targets.Israelis_Count
# #split the data, train the model and get prediction for the training and for the test  
# #######################################################################################################################
# X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.3, random_state=1357)
# train_df_scaled = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
# test_df_scaled = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)

# lm = LinearRegression()  # define our model using least square method
# lm.fit(X_train,y_train)   # Fit our linear mode
# #train
# fitted = np.exp(lm.predict(X_train))
# predicted_train = round(pd.Series(fitted, index=y_train.index, name='Predicted_train_'+targets_list_name[idx]),ndigits=2)
# orignal_train=np.exp(train_df_scaled[[targets_list_name[idx]]])
# train_df = pd.merge(left=orignal_train, right=predicted_train, left_index=True, right_index=True)
# train_df

# #test
# fitted = np.exp(lm.predict(X_test))
# predicted_test = round(pd.Series(fitted, index=y_test.index, name='Predicted_test_'+targets_list_name[idx]),ndigits=2)
# orignal_test = np.exp(test_df_scaled[[targets_list_name[idx]]])
# test_df_scaled = pd.merge(left=orignal_test, right=predicted_test, left_index=True, right_index=True)
# test_df_scaled


<IPython.core.display.Javascript object>

In [19]:
X_train, X_test, y_train, y_test = train_test_split(scaled_X, scaled_targets.Israelis_Count, test_size=0.3, random_state=312148513)
print(X_train.index)
print(y_train.index)

Int64Index([19898, 35768, 34587, 36759, 15521, 22755, 19313, 14779, 13585, 21378,
            ...
            28541, 13838,  4860, 10010, 29235,  2731,  6186, 23620, 17510, 38389], dtype='int64', length=32540)
Int64Index([19898, 35768, 34587, 36759, 15521, 22755, 19313, 14779, 13585, 21378,
            ...
            28541, 13838,  4860, 10010, 29235,  2731,  6186, 23620, 17510, 38389], dtype='int64', length=32540)


<IPython.core.display.Javascript object>

Model MLR with Pollutions droped Null rows

In [20]:
Descripton = "Model MLR with Pollutions droped Null rows"
Model_type= "MLR"

res=pd.read_excel("../../../../res.xlsx")

targets = [scaled_targets.Israelis_Count,scaled_targets.Total,scaled_targets.Tourists_Count]
targets_list_name = ['Israelis_Count','Total','Tourists_Count']
idx=0
for y in targets:
     
    #split the data, train the model and get prediction for the training and for the test  
    #######################################################################################################################
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.3, random_state=312148513)
    train_df_scaled = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
    test_df_scaled = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
    print(len(train_df_scaled))
    lm = LinearRegression()  # define our model using least square method
    lm.fit(X_train,y_train)   # Fit our linear model

    #train
    fitted = np.exp(lm.predict(X_train))
    predicted_train = round(pd.Series(fitted, index=y_train.index, name='Predicted_train_'+targets_list_name[idx]),ndigits=2)
    orignal_train=np.exp(train_df_scaled[[targets_list_name[idx]]])
    train_df = pd.merge(left=orignal_train, right=predicted_train, left_index=True, right_index=True)
    train_df

    #test
    fitted = np.exp(lm.predict(X_test))
    predicted_test = round(pd.Series(fitted, index=y_test.index, name='Predicted_test_'+targets_list_name[idx]),ndigits=2)
    orignal_test = np.exp(test_df_scaled[[targets_list_name[idx]]])
    test_df = pd.merge(left=orignal_test, right=predicted_test, left_index=True, right_index=True)
    test_df
     #######################################################################################################################



     
     
     #plot the residuals graph
     #######################################################################################################################
    
     #calculate the residuals
    test_df['residuals'] = test_df['Predicted_test_'+targets_list_name[idx]] - test_df[targets_list_name[idx]]
    train_df['residuals'] = train_df['Predicted_train_'+targets_list_name[idx]] - train_df[targets_list_name[idx]]
    
    fig= go.Figure()
    fig.add_trace(
      go.Scatter(
          x=train_df['Predicted_train_'+targets_list_name[idx]],
          y=train_df.residuals,
          mode='markers',
          name='train residuals',
          marker_color='blue',
          marker_size=1.5,
          marker_line_width=0,
       )
     )
    fig.add_trace(
      go.Scatter(
          x=test_df['Predicted_test_'+targets_list_name[idx]],
          y=test_df.residuals,
          mode='markers',
          name='test residuals',
          marker_color='red',
          marker_size=1.5,
          marker_line_width=0,
       )
     )
    fig.add_trace(
       go.Scatter(
          x=test_df['Predicted_test_'+targets_list_name[idx]],
          y=test_df.residuals*0,
          mode='lines',
          name='zero line',
          marker_color='black',
          marker_size=1.5,
          marker_line_width=0,
       )
     )
    fig.update_layout(
        title="Residuals of Predicted "+targets_list_name[idx],
        xaxis_title="Predicted "+targets_list_name[idx],
        yaxis_title="Residuals",
        font=dict(
            size=14,
            color="RebeccaPurple"
        )
    )
    # fig.show()
     #######################################################################################################################

     #create folder
     ########################################
    if not os.path.exists(Descripton):
      os.mkdir(Descripton)
    fig.write_image(Descripton+"/results_"+targets_list_name[idx]+'_.png', width=1500, height=600)
    #########################################

    #create txt file with the metrix result 
    ################################################################################################################################################     
    f = open(Descripton+"/results_"+targets_list_name[idx]+'_.txt', 'w')

    #  correct_rows = train_df.loc[abs(train_df['Predicted_train_'+targets_list_name[idx]]-train_df[targets_list_name[idx]])<=50].any(axis=1).count()
    #  ACC_Training = round(correct_rows/len(train_df),3)
    
    MSE_Training = metrics.mean_squared_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
    RMSE_Training = np.sqrt(metrics.mean_squared_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]]))
    MAE_Training = metrics.mean_absolute_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
    R2_Training=metrics.r2_score(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
    std_Training = np.std(predicted_train)

    f.write("------ TRAIN DATA ------\n")
    #f.write("Model "+' mod0' +" Accuracy: "+str(ACC_Training)+"\n")
    f.write("MSE : "+str(MSE_Training)+", RMSE: "+str(RMSE_Training)+", MAE : "+str(MAE_Training)+"\n")
    f.write("R2 TRAIN "+ str(R2_Training)+"\n")


    #  correct_rows = test_df.loc[abs(test_df['Predicted_test_'+targets_list_name[idx]]-test_df[targets_list_name[idx]])<=50].any(axis=1).count()
    #  ACC_Test = round(correct_rows/len(test_df),3)
     
    MSE_Test = metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
    RMSE_Test = np.sqrt(metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]]))
    MAE_Test = metrics.mean_absolute_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
    R2_Test = metrics.r2_score(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
    std_test = np.std(predicted_test)

    f.write("\n")
    f.write("------ TEST DATA ------\n")
    #f.write("Model "+' mod0' +" Accuracy: "+str(ACC_Test)+"\n")
    f.write("MSE : "+str(MSE_Test)+", RMSE: "+str(RMSE_Test)+", MAE : "+str(MAE_Test)+"\n")
    f.write("R2 TEST "+ str(R2_Test)+"\n")
    f.write("--------------------------------\n")

    f.close()
    ################################################################################################################################################     

    #Add to excel result the new line
    ################################################################################################################################################
    Target=targets_list_name[idx]
    Site_in_this_model='ALL'
    new_row = {
      'Descripton':Descripton,
      "Target":Target,
      'Model_type':Model_type,
      # 'Model_number':Model_number,
      'Site_in_this_model':Site_in_this_model,
      "Size Train":len(train_df),
      # 'ACC_Training':ACC_Training,
      'MAE_Training':MAE_Training,
      'MSE_Training':MSE_Training,
      'RMSE_Training':RMSE_Training,
      'STD_Training':std_Training,
      'R2_Training':R2_Training,
      "Size Test":len(test_df),
      # 'ACC_Test':ACC_Test,
      'MAE_Test':MAE_Test,
      'MSE_Test':MSE_Test,
      'RMSE_Test':RMSE_Test,
      'STD_Test':std_test,
      'R2_Test':R2_Test 
      }

    res=res.append(new_row,ignore_index=True)
    ################################################################################################################################################     
     
    # Index for the next target y
    idx+=1

     
res.to_excel("../../../../res.xlsx",index=False)


32540
32540
32540


<IPython.core.display.Javascript object>